In [3]:
from transformers import  RagRetriever, RobertaTokenizer, RobertaForQuestionAnswering, RobertaModel
from langchain.chains import RetrievalQA
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import torch
import numpy as np

/home/alina/hermes/hermes/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x78ec69ee0da0>>
Traceback (most recent call last):
  File "/home/alina/hermes/hermes/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


### Создаем данные

In [3]:
def generate_qa_pairs(f1, f2, f3):
    qa_pairs = []
    with open(f1, "r") as file1, open(f2, "r") as file2, open(f3, "r") as file3:
        for q, c, a in zip(file1, file2, file3):
            qa_pairs.append((q.strip(), c.strip(), a.strip()))
    return qa_pairs


In [26]:
qa_pairs = generate_qa_pairs("train.question", "train.code", "train.answer")[:1000]

In [27]:
docs = [f"Question: {q} Code: {c} Answer: {a} " for q, c, a in qa_pairs]


### Модель

In [31]:
model = RobertaModel.from_pretrained("microsoft/codebert-base")


In [50]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
emb = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")
vector = emb.embed_query("apple")
len(vector)

768

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
vector = model.encode("apple")
len(vector)


384

In [ ]:
#с документации
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer

# Initialize the client
client = QdrantClient(":memory:")  # or QdrantClient(path="path/to/db")
model = SentenceTransformer("all-MiniLM-L6-v2")  
vectors = model.encode(documents).tolist()

metadata = [{"lang": "python"} for _ in range(len(documents))]

points = [
    PointStruct(vector=vectors[i], payload=metadata[i]) for i in range(len(documents))
]

# Use the new add method
client.add(
    collection_name="demo_collection",
    documents=documents,
    metadata= metadata
)
client.upsert(collection_name="demo_collection", points=points)

search_result = client.query(
    collection_name="demo_collection",
    query_text="This is a query document"
)
print(search_result)

In [7]:
#загрузка векторов
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
import json
qa_pairs = generate_qa_pairs("./db/train/train.question", "./db/train/train.code", "./db/train/train.answer")
docs = [f"Question: {q} Code: {c}" for q, c, _ in qa_pairs]
answers = [f"Answer: {a}" for _, __, a in qa_pairs]
model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-мерный вектор

vectors = model.encode(docs).tolist()
with open("db_vectors.json", "w") as f:
    json.dump(vectors, f)
answers

['Answer: a suite',
 'Answer: Yes',
 'Answer: Yes',
 'Answer: a message',
 'Answer: to create the campaign',
 'Answer: Yes',
 'Answer: an explicit budget',
 'Answer: the plugins directory path',
 'Answer: Yes',
 'Answer: an image',
 'Answer: Yes',
 'Answer: parameters to url',
 'Answer: Yes',
 'Answer: for a workbook',
 'Answer: Yes',
 'Answer: an error',
 'Answer: Yes',
 'Answer: a token',
 'Answer: Yes',
 'Answer: windows',
 'Answer: an empty urlconf 404 error response',
 'Answer: Yes',
 'Answer: Yes',
 'Answer: a new des cipher',
 'Answer: a file',
 'Answer: Yes',
 'Answer: Yes',
 'Answer: its content',
 'Answer: Yes',
 'Answer: the module',
 'Answer: using the plain encoding',
 'Answer: individual index definition',
 'Answer: to protocol buffer',
 'Answer: Yes',
 'Answer: Yes',
 'Answer: the python source for a node tree',
 'Answer: Yes',
 'Answer: a given cache cluster is deleted',
 'Answer: Yes',
 'Answer: already',
 'Answer: Yes',
 'Answer: the given feedbackmessagereference',
 

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
import json
def generate_qa_pairs(f1, f2, f3):
    qa_pairs = []
    with open(f1, "r") as file1, open(f2, "r") as file2, open(f3, "r") as file3:
        for q, c, a in zip(file1, file2, file3):
            qa_pairs.append((q.strip(), c.strip(), a.strip()))
    return qa_pairs
qa_pairs = generate_qa_pairs("./db/train/train.question", "./db/train/train.code", "./db/train/train.answer")[:1000]
docs = [f"Question: {q} Code: {c} Answer: {a} " for q, c, a in qa_pairs]
client = QdrantClient(":memory:") 

model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-мерный вектор

client.create_collection(
    collection_name="demo_collection",
    vectors_config={"size": 384, "distance": "Cosine"}
)
print("Empty collection created")

print("Embedding...") #Именно эта часть выполняется долго
# vectors = model.encode(docs).tolist()

with open("db_vectors.json", "r") as f:
    vectors = json.load(f)

metadata = [{"lang": "python"} for _ in range(len(docs))]

print("Add to collection")
points = [
    PointStruct(id = i, vector=vectors[i], payload={"lang": "python", 'text': docs[i]}) for i in range(len(docs))
]
client.upsert(collection_name="demo_collection", points=points)

print("Collection of vectors is created")


#пример из тестовых данных
query_text = "What does the code create at dist_path ? def make trivial sdist dist path setup py setup py file tarfile Tar Info name 'setup py' try Mem File Bytes I Oexcept Attribute Error Mem File String I Osetup py bytes Mem File setup py encode 'utf- 8 ' setup py file size len setup py bytes getvalue dist tarfile open dist path 'w gz' try dist addfile setup py file fileobj setup py bytes finally dist close"
query_vector = model.encode(query_text).tolist()

print("Searching...")
search_result = client.query_points(collection_name="demo_collection",query=query_vector, limit = 10)


Empty collection created
Embedding...
Add to collection
Collection of vectors is created
Searching...


In [ ]:
res = [(point.payload['text'], answers[point.id]) for point in search_result.points]
res

[('Question: What does the code create ? Code: def make archive base name format root dir None base dir None verbose 0 dry run 0 owner None group None save cwd os getcwd if root dir is not None log debug "changinginto\'%s\'" root dir base name os path abspath base name if not dry run os chdir root dir if base dir is None base dir os curdirkwargs {\'dry run\' dry run}try format info ARCHIVE FORMATS[format]except Key Error raise Value Error "unknownarchiveformat\'%s\'" % format func format info[ 0 ]for arg val in format info[ 1 ] kwargs[arg] valif format \'zip\' kwargs[\'owner\'] ownerkwargs[\'group\'] grouptry filename func base name base dir **kwargs finally if root dir is not None log debug "changingbackto\'%s\'" save cwd os chdir save cwd return filename Answer: an archive file ',
  'Answer: an archive file'),
 ("Question: What does the code setup ? Code: def setup platform hass config add devices discovery info None file path config[CONF FILE PATH]if not os access file path os R OK 

In [ ]:
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_core.documents import Document

def generate_qa_pairs(f1, f2, f3):
    qa_pairs = []
    with open(f1, "r") as file1, open(f2, "r") as file2, open(f3, "r") as file3:
        for q, c, a in zip(file1, file2, file3):
            qa_pairs.append((q.strip(), c.strip(), a.strip()))
    return qa_pairs
qa_pairs = generate_qa_pairs("./db/train/train.question", "./db/train/train.code", "./db/train/train.answer")
docs = [f"Question: {q} Code: {c}" for q, c, a in qa_pairs]
answers = [f"Answer: {a}" for _, __, a in qa_pairs]


client = QdrantClient(path = "./qdrant") 

model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # 384-мерный вектор

client.create_collection(
    collection_name="disk_collection",
    vectors_config={"size": 384, "distance": "Cosine"}
)
print("Empty collection created")

print("Embedding...") 
db = QdrantVectorStore(
    client = client, 
    embedding=model,
    collection_name="disk_collection"
)
documents = [Document(page_content=i, metadata = {"lang": "python"}) for i in docs]


print("Collection of vectors is created")

print("Add documents")

db.add_documents(documents=documents, ids = [id for id in range(len(docs))])
print("documents added")


Empty collection created
Embedding...
Collection of vectors is created
Add documents


/home/alina/hermes/hermes/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:444: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 20032 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  self.client.upsert(


documents added


### отсюда

In [5]:
def generate_qa_pairs(f1, f2, f3):
    qa_pairs = []
    with open(f1, "r") as file1, open(f2, "r") as file2, open(f3, "r") as file3:
        for q, c, a in zip(file1, file2, file3):
            qa_pairs.append((q.strip(), c.strip(), a.strip()))
    return qa_pairs

In [ ]:
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
client = QdrantClient(path = "/home/alina/hermes/qdrant")
model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
db = QdrantVectorStore(
    client = client, 
    collection_name="disk_collection", 
    embedding=model
)
qa_pairs = generate_qa_pairs("./db/train/train.question", "./db/train/train.code", "./db/train/train.answer")
answers = [f"Answer: {a}" for _, __, a in qa_pairs]


/tmp/ipykernel_23346/166009726.py:4: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <disk_collection> contains 56085 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client = QdrantClient(path = "./qdrant")


RuntimeError: Storage folder ./qdrant is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [9]:
#пример из тестовых данных
query_text = "How does the code add a user in the given buckets default object access control list ? def add bucket default owner bucket name user email storage client storage Client bucket storage client bucket bucket name bucket acl reload bucket default object acl user user email grant owner bucket default object acl save print ' Addeduser{}asanownerinthedefaultaclonbucket{} ' format user email bucket name"
print("Searching...")
search_result = db.similarity_search_with_score(query=query_text, k = 5)

Searching...


In [ ]:
for i, score in search_result:
    print(score)
    print(answers[i.metadata['_id']], i.metadata['_id'])
    print(i.page_content)

Answer: on the given bucket 14485
Question: Where does the code add a user ? Code: def add bucket owner bucket name user email storage client storage Client bucket storage client bucket bucket name bucket acl reload bucket acl user user email grant owner bucket acl save print ' Addeduser{}asanowneronbucket{} ' format user email bucket name
Answer: a user 14484
Question: What does the code add on the given bucket ? Code: def add bucket owner bucket name user email storage client storage Client bucket storage client bucket bucket name bucket acl reload bucket acl user user email grant owner bucket acl save print ' Addeduser{}asanowneronbucket{} ' format user email bucket name
Answer: a user 50600
Question: What does the code remove from the access control list of the given buckets default object access control list ? Code: def remove bucket default owner bucket name user email storage client storage Client bucket storage client bucket bucket name bucket acl reload bucket default object a

In [102]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd

In [103]:
qwen = pd.DataFrame(columns=['BLEU1', 'BLEU4'])
qwen_rag = pd.DataFrame(columns=['BLEU1', 'BLEU4'])
weights_4 = (0.25, 0.25, 0.25, 0.25)
weights_1 = (1.0, 0.0, 0.0, 0.0)
smooth = SmoothingFunction().method2
with open('qwen.txt', 'r') as f1, open('qwen_rag.txt', 'r') as f2, open('./db/test/test.answer', 'r') as f3:
        for q, rag, orig in zip(f1, f2, f3):
                candidate = q.split()
                candidate_rag = rag.split()
                orig = [orig.split()]
                qwen.loc[len(qwen)] = [sentence_bleu(orig, candidate, weights=weights_4, smoothing_function=smooth), sentence_bleu(orig, candidate, weights=weights_1, smoothing_function=smooth)]
                qwen_rag.loc[len(qwen_rag)] = [sentence_bleu(orig, candidate_rag, weights=weights_4, smoothing_function=smooth), sentence_bleu(orig, candidate_rag, weights=weights_1, smoothing_function=smooth)]
                

In [81]:
qwen.sum()/qwen.shape[0]

np.float64(0.08795774945253818)

In [77]:
qwen_rag.sum()/qwen_rag.shape[0]

np.float64(0.2065410913900342)

In [99]:
qwen_rag

,BLEU1,BLEU4
0,0.408248,0.333333
1,0.594604,1.000000
2,0.143252,0.214879
3,0.408248,0.333333
4,0.000000,0.000000
...,...,...
5995,0.000000,0.000000
5996,0.028367,0.023161
5997,0.257453,0.477688
5998,0.150908,0.118092


In [100]:
qwen

,BLEU1,BLEU4
0,0.144907,0.222222
1,0.319472,0.250000
2,0.191820,0.330936
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
5995,0.000000,0.000000
5996,0.098228,0.090484
5997,0.196407,0.250000
5998,0.065230,0.105263


In [2]:
from langdetect import detect

def is_english_and_short(text: str) -> bool:
    if len(text.split()) > 120:
        return False
    return detect(text) == 'en'

In [3]:
is_english_and_short("The function appears to test a custom 400 Bad Request template, but it lacks clarity and violates Python best practices by not adhering to PEP 8 naming conventions (e.g., `test custom bad request template` should be `test_custom_bad_request_template`). Additionally, the function's logic is incomplete and unclear without proper syntax and context.")

True